In [ ]:
!pip install aquacrop

from google.colab import output
output.clear()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !git clone https://github.com/aquacropos/aquacrop.git
# !cd /content/aquacrop
%pip install -r requirements.txt


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
!pip install numba>=0.56.0 numpy==1.22.4 pandas>=1.3.5 tqdm>=4.64.0 cffi==1.16.0 matplotlib seaborn


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.16 requires numpy>=1.24, but you have numpy 1.22.4 which is incompatible.
albumentations 1.4.15 requires numpy>=1.24.4, but you have numpy 1.22.4 which is incompatible.
arviz 0.19.0 requires numpy>=1.23.0, but you have numpy 1.22.4 which is incompatible.
astropy 6.1.4 requires numpy>=1.23, but you have numpy 1.22.4 which is incompatible.
bigframes 1.21.0 requires numpy>=1.24.0, but you have numpy 1.22.4 which is incompatible.
chex 0.1.87 requires numpy>=1.24.1, but you have numpy 1.22.4 which is incompatible.
cudf-cu12 24.6.1 requires numpy<2.0a0,>=1.23, but you have numpy 1.22.4 which is incompatible.
ibis-framework 9.2.0 requires numpy<3,>=1.23.2, but you have numpy 1.22.4 which is incompatible.
jax 0.4.33 requires numpy>=1.24, but you have numpy 1.22.4 which is incompatible.
jaxlib 0.4.33 requires 

In [ ]:
!pip install python-dotenv
!pip install numpy==1.23.0
!pip install pandas==2.1.4 scipy==1.10.1 cffi==1.16.0 matplotlib==3.5.0

  Using cached numpy-1.23.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.2 kB)
Using cached numpy-1.23.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.16 requires numpy>=1.24, but you have numpy 1.23.0 which is incompatible.
albumentations 1.4.15 requires numpy>=1.24.4, but you have numpy 1.23.0 which is incompatible.
aquacrop 3.0.9 requires numpy==1.22.4, but you have numpy 1.23.0 which is incompatible.
bigframes 1.21.0 requires numpy>=1.24.0, but you have numpy 1.23.0 which is incompatible.
chex 0.1.87 requires numpy>=1.24.1, but you have numpy 1.23.0 which is incompatible.
ibis-framework 9.2.0 requires

  Using cached matplotlib-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached matplotlib-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.9 MB)
ERROR: Operation cancelled by user


In [ ]:
import os
os.environ['DEVELOPMENT'] = 'True'

In [ ]:
from aquacrop import AquaCropModel, Soil, Crop, InitialWaterContent, FieldMngt, GroundWater
from aquacrop.utils import prepare_weather, get_filepath
import pandas as pd
import os
import requests
import xml.etree.ElementTree as ET
import math
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()
os.environ['DEVELOPMENT'] = 'True'
API_KEY=os.getenv("API_KEY")


class WeatherDataFetcher:
    def __init__(self, location_name: str, start_date: str, end_date: str, station_id: str):
        self.location_name = location_name
        self.start_date = start_date
        self.end_date = end_date
        self.station_id = station_id
        self.lat = 36.1081929

    # Hargreaves 방정식 정의
    @staticmethod
    def hargreaves_evapotranspiration(tmax, tmin, tmean, lat, doy):
        Gsc = 0.0820  # 태양 상수 (MJ/m^2/min)
        phi = math.radians(lat)  # 위도를 라디안으로 변환
        delta = 0.409 * math.sin((2 * math.pi / 365) * doy - 1.39)  # 일 년 중 태양 각도
        omega_s = math.acos(-math.tan(phi) * math.tan(delta))  # 일출과 일몰 각도
        Ra = (24 * 60 / math.pi) * Gsc * (omega_s * math.sin(phi) * math.sin(delta) + math.cos(phi) * math.cos(delta) * math.sin(omega_s))  # 일사량
        ET0 = 0.0023 * Ra * ((tmax - tmin) ** 0.5) * (tmean + 17.8)  # Hargreaves 공식
        return ET0

    # 날짜를 day of year로 변환
    @staticmethod
    def get_day_of_year(date_str):
        date = datetime.strptime(date_str, "%Y-%m-%d")
        return date.timetuple().tm_yday

    # API URL 생성
    def construct_api_url(self):
        return (f"http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=AZU%2FN4raq67jaDV74XFu0Xuqoutik8v3sRxmnlNjJMiX6pGOqhXBOuc1QZbdEc2ahF52jkKAo1H8oP7jN%2BsFvQ%3D%3D"
                f"&numOfRows=100&pageNo=1&dataCd=ASOS&dateCd=DAY&startDt={self.start_date}&endDt={self.end_date}&stnIds={self.station_id}")

    # XML 데이터를 처리하여 weather_df 생성
    def fetch_weather_data(self):
        api_url = self.construct_api_url()

        # 기상청 API에서 XML 데이터를 가져옴
        response = requests.get(api_url)
        root = ET.fromstring(response.content)

        weather_records = []

        # XML 데이터에서 필요한 정보를 추출
        for item in root.findall(".//item"):
            date_str = item.find("tm").text  # 날짜
            tmin = float(item.find("minTa").text)  # 최저 기온
            tmax = float(item.find("maxTa").text)  # 최고 기온
            tmean = float(item.find("avgTa").text)  # 평균 기온
            precipitation = float(item.find("sumRn").text) if item.find("sumRn").text else 0.0  # 강수량

            doy = self.get_day_of_year(date_str)  # 일 년 중 일(day of year)

            # Hargreaves 방정식으로 Reference ET 계산
            reference_et = self.hargreaves_evapotranspiration(tmax, tmin, tmean, self.lat, doy)

            # DataFrame을 위해 각 데이터를 딕셔너리로 저장
            weather_records.append({
                "MinTemp": tmin,
                "MaxTemp": tmax,
                "Precipitation": precipitation,
                "ReferenceET": reference_et,
                "Date": date_str
            })

        # Pandas DataFrame으로 변환
        weather_df = pd.DataFrame(weather_records)
        return weather_df

# 사용 예시
if __name__ == "__main__":
    location = "Gumi"
    start_date = "20230101"
    end_date = "20231231"
    station_id = "279"

    weather_fetcher = WeatherDataFetcher(location, start_date, end_date, station_id)
    weather_df = weather_fetcher.fetch_weather_data()

    print(weather_df.head())

   MinTemp  MaxTemp  Precipitation  ReferenceET        Date
0     -5.9      7.4            0.0     2.426742  2023-01-01
1     -4.4      4.1            0.0     1.830227  2023-01-02
2     -5.3      4.4            0.0     1.916932  2023-01-03
3     -6.0      6.7            0.0     2.228079  2023-01-04
4     -7.0      7.1            0.0     2.316123  2023-01-05


In [ ]:
import json
from aquacrop import Crop, AquaCropModel, Soil
import pandas as pd

# JSON 파일에서 작물 파라미터를 읽어오는 함수
def load_crop_parameters(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# JSON 파일 경로
file_path = 'crop_parameter.json'

# 작물 파라미터 불러오기
crop_parameters = load_crop_parameters(file_path)

# 토마토 파라미터 가져오기
tomato_params = crop_parameters.get('tomato')

# Crop 객체 생성 (제공된 모든 파라미터 사용)
# Crop 객체 생성 (제공된 모든 파라미터 사용)
custom_tomato_crop = Crop(
    c_name='custom',
    name=tomato_params['Name'],
    planting_date=tomato_params['PlantingDate'],
    harvest_date=tomato_params['HarvestDate'],
    crop_type=tomato_params['CropType'],
    plant_method=tomato_params['PlantMethod'],
    CalendarType=tomato_params['CalendarType'],
    switch_gdd=tomato_params['SwitchGDD'],
    emergence=tomato_params['Emergence'],
    max_rooting=tomato_params['MaxRooting'],
    senescence=tomato_params['Senescence'],
    maturity=tomato_params['Maturity'],
    hi_start=tomato_params['HIstart'],
    flowering=tomato_params['Flowering'],
    yld_form=tomato_params['YldForm'],
    gdd_method=tomato_params['GDDmethod'],
    t_base=tomato_params['Tbase'],
    t_upp=tomato_params['Tupp'],
    pol_heat_stress=tomato_params['PolHeatStress'],
    tmax_up=tomato_params['Tmax_up'],
    tmax_lo=tomato_params['Tmax_lo'],
    pol_cold_stress=tomato_params['PolColdStress'],
    tmin_up=tomato_params['Tmin_up'],
    tmin_lo=tomato_params['Tmin_lo'],
    tr_cold_stress=tomato_params['TrColdStress'],
    Zmin=tomato_params['Zmin'],
    Zmax=tomato_params['Zmax'],
    fshape_r=tomato_params['fshape_r'],
    SxTopQ=tomato_params['SxTopQ'],
    SxBotQ=tomato_params['SxBotQ'],
    PlantPop=tomato_params['PlantPop'],
    ccx=tomato_params['CCx'],
    cco=tomato_params['CCo'],
    cgc=tomato_params['CGC'],
    cdc=tomato_params['CDC'],
    kcb=tomato_params['Kcb'],
    fage=tomato_params['fage'],
    wp=tomato_params['WP'],
    hi0=tomato_params['HI0'],
    p_up1=tomato_params['p_up1'],
    p_up2=tomato_params['p_up2'],
    p_up3=tomato_params['p_up3'],
    p_up4=tomato_params['p_up4'],
    p_lo1=tomato_params['p_lo1'],
    p_lo2=tomato_params['p_lo2'],
    p_lo3=tomato_params['p_lo3'],
    p_lo4=tomato_params['p_lo4'],
    fshape_w1=tomato_params['fshape_w1'],
    fshape_w2=tomato_params['fshape_w2'],
    fshape_w3=tomato_params['fshape_w3'],
    fshape_w4=tomato_params['fshape_w4'],
    SeedSize=tomato_params['SeedSize'],
    Determinant=tomato_params['Determinant'],
    HIstartCD=tomato_params['HIstartCD'],
    FloweringCD=tomato_params['FloweringCD'],
)


In [ ]:
sandy_loam = Soil(soil_type='SandyLoam')
InitWC = InitialWaterContent(value=['FC'])

In [ ]:
weather_df['Date'] = pd.to_datetime(weather_df['Date'])
sim_end_time = weather_df['Date'].max().strftime('%Y/%m/%d')

# 'Date' 열을 datetime 형식으로 변환 (이미 처리되었을 수 있음)
weather_df['Date'] = pd.to_datetime(weather_df['Date'])

# 'MinTemp'와 'MaxTemp' 열을 float 형식으로 변환 (숫자가 아닌 값이 있을 경우 오류가 발생할 수 있음)
weather_df['MinTemp'] = pd.to_numeric(weather_df['MinTemp'], errors='coerce')
weather_df['MaxTemp'] = pd.to_numeric(weather_df['MaxTemp'], errors='coerce')

# NaN 값이 있는지 확인
print(weather_df.isna().sum())

# weather_df.reset_index('Date', inplace=True)
# 필요시 NaN 값 채우기 (예: 선형 보간)
weather_df['MinTemp'].fillna(method='ffill', inplace=True)
weather_df['MaxTemp'].fillna(method='ffill', inplace=True)

# 데이터 확인
print(weather_df.head())

model = AquaCropModel(
    sim_start_time="2023/01/01",
    sim_end_time=sim_end_time,
    weather_df=weather_df,
    soil=sandy_loam,
    crop=custom_tomato_crop,  # 올바른 Crop 객체 사용
    initial_water_content=InitWC
)

model.run_model(till_termination=True)

MinTemp          0
MaxTemp          0
Precipitation    0
ReferenceET      0
Date             0
dtype: int64
   MinTemp  MaxTemp  Precipitation  ReferenceET       Date
0     -5.9      7.4            0.0     2.426742 2023-01-01
1     -4.4      4.1            0.0     1.830227 2023-01-02
2     -5.3      4.4            0.0     1.916932 2023-01-03
3     -6.0      6.7            0.0     2.228079 2023-01-04
4     -7.0      7.1            0.0     2.316123 2023-01-05


<ipython-input-38-3c2caa163430>:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  weather_df['MinTemp'].fillna(method='ffill', inplace=True)
<ipython-input-38-3c2caa163430>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  weather_df['MaxTemp'].fillna(method='ffill', inplace=True)
/usr/local/lib/python3.10/dist-packages/aquacrop/entities/soil.py:363: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.profile = self.profile.fillna(method="ffill")


AttributeError: 'Crop' object has no attribute 'HIstartCD'

In [ ]:
model.run_model(till_termination=True)

/usr/local/lib/python3.10/dist-packages/aquacrop/entities/soil.py:363: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.profile = self.profile.fillna(method="ffill")


AttributeError: 'dict' object has no attribute 'Zmax'

In [ ]:
model._outputs.water_flux.head(100)
# model._outputs.water_storage.head()
# model._outputs.crop_growth.head()
# model._outputs.final_stats.head()

,time_step_counter,season_counter,dap,Wr,z_gw,surface_storage,IrrDay,Infl,Runoff,DeepPerc,CR,GwIn,Es,EsPot,Tr,TrPot
0,0.0,0.0,1.0,60.15,NaN,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,5.848361,9.623310,0.000000,0.000000
1,1.0,0.0,2.0,70.57,NaN,0.0,0.0,15.710389,0.089611,0.000000,0.0,0.0,5.290432,5.290432,0.000000,0.000000
2,2.0,0.0,3.0,108.67,NaN,0.0,0.0,48.252430,27.747570,3.133293,0.0,0.0,6.394370,6.394370,0.000000,0.000000
3,3.0,0.0,4.0,71.06,NaN,0.0,0.0,11.449234,0.350766,38.215533,0.0,0.0,7.044152,7.044152,0.000000,0.000000
4,4.0,0.0,5.0,62.44,NaN,0.0,0.0,0.100000,0.000000,7.738695,0.0,0.0,4.399689,6.206008,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95.0,0.0,96.0,317.93,NaN,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.223411,0.296186,0.000000,0.000000
96,96.0,0.0,97.0,317.71,NaN,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.220793,0.304671,0.000000,0.000000
97,97.0,0.0,98.0,321.22,NaN,0.0,0.0,3.900000,0.000000,0.000000,0.0,0.0,0.369811,0.369811,0.013039,0.013039
98,98.0,0.0,99.0,321.03,NaN,0.0,0.0,0.100000,0.000000,0.000000,0.0,0.0,0.259896,0.264111,0.028172,0.028172


In [ ]:
# 1. 겨울철 잔류 수분 설정
initial_winter_water_content = InitialWaterContent(wc_type='Pct', value=[40])  # 1월 1일을 기준으로 초기 수분 40%

# 2. 작물 시작일 전까지 기상 데이터 기반 초기 수분 설정
def calculate_initial_water_content(weather_data, soil):
    # 기상 데이터에서 강수량과 증발산량을 추출해 토양에 잔류하는 수분 계산
    total_precipitation = weather_data['Precipitation'].sum()
    total_evapotranspiration = weather_data['ReferenceET'].sum()
    water_content = soil.initial_moisture + (total_precipitation - total_evapotranspiration)

    # 잔류 수분 계산 후 초기 값 설정
    return InitialWaterContent(wc_type='Pct', value=[water_content])

# 예시: 1월 1일부터 3월 31일까지 기상 데이터를 사용해 초기 수분 설정
initial_water_content = calculate_initial_water_content(winter_weather_df, loam_soil)

# AquaCrop 모델 설정
# model = AquaCropModel(
#     sim_start_time='2023/04/01',
#     sim_end_time='2023/10/30',
#     weather_df=weather_df,
#     soil=loam_soil,
#     crop=tomato,
#     initial_water_content=initial_water_content,
#     irrigation_management=irrmngt
# )


Date             datetime64[ns]
MinTemp                 float64
MaxTemp                 float64
Precipitation           float64
ReferenceET             float64
dtype: object
